In [1]:
%run /Users/maayanlab/Documents/init.ipy
os.chdir('..')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


### Infiles

In [2]:
infile = 'rawdata/prem_prostate-combined_result_table_druggable.txt'

### Computation

In [3]:
# Read data
combined_dataframe = pd.read_table(infile)
combined_dataframe = combined_dataframe[combined_dataframe['selected']].sort_values('nes', ascending=True)
combined_dataframe.head()

,entrez_id,gene_symbol,logfc,nes,moa,combined,selected,null,drugs,drug_labels
61,79898,ZNF613,-2.026473,-2.775676,3.656572,-0.622887,True,True,0,NaN
62,79921,TCEAL4,-0.681810,-2.745105,1.963373,0.552768,True,True,0,NaN
8,10481,HOXB13,-0.864521,-2.589535,10.152984,-5.348167,True,False,0,NaN
34,3664,IRF6,-1.511285,-2.516363,11.132734,-6.092694,True,False,0,NaN
51,57630,SH3RF1,-3.157963,-2.424530,6.518297,-2.894730,True,False,0,NaN


##### Combined Results

In [4]:
combined_dataframe[combined_dataframe['selected']].to_csv('data/combined_results.txt', index=False)

##### Genes

In [205]:
gene_dataframe = combined_dataframe['gene_symbol'].to_frame()
gene_dataframe['len'] = 11
gene_dataframe['id'] = gene_dataframe['gene_symbol']
gene_dataframe['label'] = gene_dataframe['gene_symbol']
gene_dataframe['color'] = "white"
gene_dataframe.head()

,gene_symbol,len,id,label,color
887,ZNF613,11,ZNF613,ZNF613,white
977,TCEAL4,11,TCEAL4,TCEAL4,white
81,HOXB13,11,HOXB13,HOXB13,white
57,IRF6,11,IRF6,IRF6,white
73,WFS1,11,WFS1,WFS1,white


In [234]:
with open('data/genes.json', 'w') as openfile:
    openfile.write(json.dumps(gene_dataframe.drop('gene_symbol', axis=1).to_dict(orient='records'), indent=4))

##### PrePPI

In [35]:
%R load('./rawdata/preppi2013.rda')
%Rpull preppi

In [60]:
preppi.to_csv('data/preppi.txt', sep='\t', index=False)

In [146]:
# Filter
preppi_filtered = preppi.replace('-', np.nan).dropna()
preppi_filtered['EntID_a'] = preppi_filtered['EntID_a'].astype('int')
preppi_filtered['EntID_b'] = preppi_filtered['EntID_b'].astype('int')
preppi_filtered = pd.concat([preppi_filtered, preppi_filtered.rename(columns={'EntID_a': 'sEntID_a', 'EntID_b': 'sEntID_b'}).rename(columns={'sEntID_b':'EntID_a', 'sEntID_a':'EntID_b'})])
preppi_filtered.head()


,EntID_a,EntID_b,prob
1,54502,54502,0.49
2,79733,54502,0.07
3,79733,79733,0.41
4,55236,55236,0.37
5,57488,54502,0.03


In [147]:
# Get PrePPI
gene_dataframe = combined_dataframe.loc[combined_dataframe['selected'], ['gene_symbol', 'entrez_id']]
gene_dataframe['entrez_id'] = gene_dataframe['entrez_id'].astype('int')
gene_dataframe.head()

,gene_symbol,entrez_id
4,CDH1,999
1,PTTG1,9232
3,CDK4,1019
57,IRF6,3664
7,TOP2A,7153


In [148]:
interaction_dataframe = preppi_filtered.merge(gene_dataframe, left_on='EntID_a', right_on='entrez_id', how='inner').rename(columns={'gene_symbol': 'gene_symbol_a'}).merge(gene_dataframe, left_on='EntID_b', right_on='entrez_id', how='inner').rename(columns={'gene_symbol': 'gene_symbol_b'})[['gene_symbol_a', 'gene_symbol_b', 'prob']].drop_duplicates()
interaction_dataframe.head()

,gene_symbol_a,gene_symbol_b,prob
0,UBE2C,UBE2C,1.00
2,CHEK1,UBE2C,0.77
3,NDC80,UBE2C,0.70
4,AURKA,UBE2C,0.87
5,ARHGAP6,UBE2C,0.17


In [220]:
interaction_dataframe = interaction_dataframe.copy()
interaction_dataframe['a_start'] = 4
interaction_dataframe['a_end'] = 6
interaction_dataframe['b_start'] = 4
interaction_dataframe['b_end'] = 6
interaction_dataframe['prob'] = [np.nan if rowData['gene_symbol_a']==rowData['gene_symbol_b'] else rowData['prob'] for index, rowData in interaction_dataframe.iterrows()]
interaction_dataframe = interaction_dataframe[['gene_symbol_a', 'a_start', 'a_end', 'gene_symbol_b', 'b_start', 'b_end', 'prob']].query('prob>0.9')
print interaction_dataframe.shape
interaction_dataframe.head()

(134, 7)


,gene_symbol_a,a_start,a_end,gene_symbol_b,b_start,b_end,prob
6,BUB1B,4,6,UBE2C,4,6,1.00
20,CKS2,4,6,UBE2C,4,6,0.97
58,BUB1B,4,6,CHEK1,4,6,1.00
64,PCNA,4,6,CHEK1,4,6,0.99
66,CCNA2,4,6,CHEK1,4,6,0.98


In [221]:
interaction_dataframe.to_csv('./data/preppi_predictions.txt', index=False)

##### Position

In [270]:
gene_correlation_dataframe = pd.pivot_table(interaction_dataframe, index='gene_symbol_a', columns='gene_symbol_b', values='prob').fillna(0).corr()
for gene_symbol in gene_dataframe['gene_symbol']:
    if gene_symbol not in gene_correlation_dataframe.columns:
        gene_correlation_dataframe[gene_symbol] = [random.random() for x in range(len(gene_correlation_dataframe.index))]
        gene_correlation_dataframe.loc[gene_symbol] = [random.random() for x in range(len(gene_correlation_dataframe.columns))]
np.fill_diagonal(gene_correlation_dataframe.values, 0)

for i in gene_correlation_dataframe.index:
    gene_correlation_dataframe.loc[i] = gene_correlation_dataframe.loc[i]+np.random.normal(0,1.1,len(gene_correlation_dataframe.index))
    
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca.fit(gene_correlation_dataframe)

pc_df = pd.Series(index=gene_correlation_dataframe.index, data=pca.components_[0]).rename('pc1').sort_values().to_frame()
sorted_genes = pc_df.index.tolist()
pc_df.head()

with open('data/genes.json', 'w') as openfile:
    openfile.write(json.dumps(gene_dataframe.set_index('gene_symbol').loc[sorted_genes].to_dict(orient='records'), indent=4))
gene_correlation_dataframe.head()

gene_symbol_b,ARHGAP6,ASF1B,AURKA,BUB1B,CCNA2,CDK2,CDK4,CENPF,CHEK1,CKS2,...,DEPDC1,BRIP1,CENPK,TMPO,PHTF2,EZH2,PTTG1,SPAG5,RACGAP1,ATAD2
gene_symbol_b,,,,,,,,,,,,,,,,,,,,,
ARHGAP6,0.953143,1.498573,0.094156,1.744418,0.629579,0.312985,0.334917,-0.073231,0.139364,0.790852,...,1.965648,0.271002,2.063105,-2.112843,-1.275408,1.485325,0.815959,0.316465,1.153234,0.020715
ASF1B,0.066977,-0.206971,0.672853,1.014477,0.137321,0.159817,-0.048134,2.352372,1.400957,1.259876,...,1.254568,0.894785,-0.644663,-0.284024,-0.850976,-0.313718,-1.457859,-0.023838,0.602711,0.276368
AURKA,-0.197975,-0.514296,0.126037,-0.037838,1.776414,0.634710,0.095992,-2.411787,0.329076,-0.533269,...,0.617136,1.528839,1.025082,-0.195862,0.777232,0.979037,0.540651,0.812170,-0.140928,-0.184275
BUB1B,1.812513,0.934801,0.256805,0.981835,-0.732948,0.187817,-0.075188,-1.311472,1.494599,0.555260,...,0.368606,0.940482,-0.807120,1.308208,0.003224,1.255579,-0.592971,2.808769,2.189124,1.784521
CCNA2,-0.104291,0.423547,-0.138835,-0.370545,-2.017004,0.084106,-0.004705,-0.304591,-0.316175,-0.278780,...,-0.634049,-0.019848,1.013414,0.598073,2.403273,1.392028,0.588767,1.494880,1.051258,-0.105044


In [268]:
var data = [
    // sourceId, sourceStart, sourceEnd, targetId, targetStart, targetEnd
    ['january', 1, 12, 'april', 18, 20],
    ['february', 20, 28, 'december', 1, 13],
];

SyntaxError: invalid syntax (<ipython-input-268-372b5bd3996a>, line 1)

##### Similarity